In [6]:
from prelu_cnn import CNN

from datasets import load_from_disk

# Load training dataset
dataset_path = "../processed_datasets/imagenet_processor"
ds = load_from_disk(dataset_path)
ds





Loading dataset from disk:   0%|          | 0/1550 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/61 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 1280967
    })
    validation: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 50000
    })
})

In [7]:
ds = ds['train'].select(range(1000))
ds['pixel_values'].shape, type(ds['pixel_values'])

(torch.Size([1000, 3, 224, 224]), torch.Tensor)

In [8]:
inputs = ds['pixel_values']
labels = ds['labels']

In [9]:
import torch
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move model and inputs to GPU if available
model = CNN(use_prelu=False, use_builtin_conv=True).to(device)

# Move tensors to the same device as the model
inputs = inputs.to(device)
labels = labels.to(device)

outputs = model(inputs)
outputs

Using device: cuda


tensor([[ 0.1230,  0.6148,  0.0310,  ...,  0.0144,  0.2355,  0.3013],
        [-0.5012,  0.5038, -0.2886,  ...,  0.2321, -0.0557, -0.4637],
        [ 0.2678,  0.4043, -0.4985,  ...,  0.0567,  0.1876,  0.6038],
        ...,
        [ 0.1555,  0.4504, -0.7211,  ..., -0.4138,  0.0079,  0.3764],
        [ 0.5574,  0.0928,  0.0194,  ..., -0.4015, -0.3036,  0.7183],
        [ 0.0968,  0.8129,  0.6444,  ...,  0.0357,  0.3662,  0.9774]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [10]:
import numpy as np

# axis=1 means we're finding the maximum value along the second dimension (columns)
# For a 2D tensor with shape (batch_size, num_classes), axis=1 gives us the class index
# with the highest probability for each sample in the batch
predictions = np.argmax(outputs.detach().cpu().numpy(), axis=1)
print("Predictions:", predictions)
print("Labels:", labels.cpu().numpy())
print("Correct predictions:", (predictions == labels.cpu().numpy()).sum(), "out of", len(predictions))


Predictions: [ 82 833 186 429 390 356 458 658 687 317  61 864 630  73 951 664 248 617
 478 630 356 830 658 141  24 703 648 355  69 899 830 797 355 439 492 191
 362 217 213 899 443 903 492 450 124 934 138 479 995 134 503 428 617 687
 699 554 469 577 679 283 420 630 390 782 296 750 665 414  61 275 750 142
 554 612 750 565 844 687 892 163 924  24 469 470 507 414 152 469 699 830
 100 775 285  19 149 443 414 395 679 701 778  24 950 107 974 554 836  24
  73 679 630 353 487 116 782 451 871 699 443 833 443 750 315 932 269 483
 497 714 959 554 836 356 679 578 658 378 296 671 283 719 384  82 546 998
 492 469 325 539 390 469 964 194 679 714 211  24 215 932 714 612 138 414
 395 390 186 899 614 296 414  61 799 263 256 999 859 138 161 799  53 729
 915 630 714 662 488 630 714 182 443 443 492 202 469 439 481 554  24  24
 577 577 668 612 687 283 325 456 443 830 428 577 194  61 395 626 142 714
 459 492 799 469 284 714 995  61   5 265 945 957   3 414 507 362 687  24
 578  83 577 450  24  24 732 289  24 7

In [11]:
import torch.nn as nn

loss_fct = nn.CrossEntropyLoss()
loss = loss_fct(outputs, labels)
loss

tensor(6.9875, device='cuda:0', grad_fn=<NllLossBackward0>)